In [1]:
import netCDF4 as nc
import pyproj
import numpy as np
import xarray as xr

# Variables

In [2]:
'''
TAIR

WRF Variable Name ---> T2
Description ---> Temperature at 2m
Units [WRF] ---> K
Units [SnowModel] ---> C
Derivation / Formula ---> C = K - 273.15 [implemented in SnowModel]


'''

'\nTAIR\n\nWRF Variable Name ---> T2\nDescription ---> Temperature at 2m\nUnits [WRF] ---> K\nUnits [SnowModel] ---> C\nDerivation / Formula ---> C = K - 273.15 [implemented in SnowModel]\n\n\n'

In [3]:
class PreprocessNetCDF(object):
    
    def __init__(self,varname,wateryear,IsHourly):
        self.varname = varname
        self.wateryear = wateryear
        self.IsHourly = IsHourly
        self.wrf_dir = '/glade/collections/rda/data/ds612.0/CTRL/'
        self.wrf_starting_name = 'wrf2d_d01_CTRL_'

        self.month_lst = []
        
        ## acceptable lists of input values
        self.acceptablevars = ["PREC_ACC_NC","T2","U10","V10","Q2","PSFC"]
        self.acceptablewy = list(range(2000,2014,1))
        self.acceptablehourly = [0,1]
        
        ## call check vars
        self.checkVars()
        
        ## month list
        self.month_lst = [(str(self.wateryear-1),'07'),
                           (str(self.wateryear-1),'08'),
                           (str(self.wateryear-1),'09'),
                           (str(self.wateryear-1),'10'),
                           (str(self.wateryear-1),'11'),
                           (str(self.wateryear-1),'12'),
                           (str(self.wateryear),'01'),
                           (str(self.wateryear),'02'),
                           (str(self.wateryear),'03'),
                           (str(self.wateryear),'04'),
                           (str(self.wateryear),'05'),
                           (str(self.wateryear),'06'),
                           (str(self.wateryear),'07'),
                           (str(self.wateryear),'08'),
                           (str(self.wateryear),'09'),
                           (str(self.wateryear),'10'),
                           (str(self.wateryear),'11'),
                           (str(self.wateryear),'12')
                         ]
    
    def checkVars(self):
        ## check WRF variable input
        check = True
        while check == True:
            if self.varname not in self.acceptablevars:
                self.varname = input('Invalid Input, Please indicate variable you would like to output.\n["PREC_ACC_NC","T2","U10","V10","Q2","PSFC"]')
            else:
                check = False
        print('')
        
        ## check water year input
        check = True     
        while check == True:
            if self.wateryear not in self.acceptablewy:
                self.wateryear = int(input('Invalid Input, Please indicate water year you would like to output for the following options.\n [2000 - 2013]'))
            else:
                check = False
                
        ## check hourly input       
        check = True
        while check == True:
            if self.IsHourly not in self.acceptablehourly:
                self.IsHourly = int(input('Invalid Input, Please indicate appropriate timestep you would like to output for the following options.\n [0 - 3hrly ; 1 - hrly]'))
            else:
                check = False
                
    def concatenateNetCDF(self):
        ds_lst = []
        for i in range(0,len(self.month_lst),3):
            fname = self.wrf_dir + self.month_lst[i][0] + '/' + self.wrf_starting_name + self.varname + '_'  + self.month_lst[i][0] + self.month_lst[i][1]+ '-' + self.month_lst[i][0] + self.month_lst[i+2][1] + '.nc'
            print(fname)
            ds = xr.open_dataset(fname)
            ds_lst.append(ds)
            
        combined_ds = xr.concat(objs = ds_lst ,dim = 'Time') ## works
        return combined_ds
        print('Finished Concat')
        if self.IsHourly == 0:
            if self.varname == "PREC_ACC_NC":
                resample = combined_ds.resample(Time="3H").sum(dim="Time") # sum precip
                ## reindex array
                idx_arr = resample[self.varname][:,:,:].values
                resample[self.varname][1:,:,:] = idx_arr[:-1,:,:]
                # set first index = 0
                resample[self.varname][0,:,:] = 0.0
            else:
                resample = combined_ds.resample(Time="3H").mean(dim="Time")
                ## reindex array
                idx_arr = resample[self.varname][:,:,:].values
                resample[self.varname][1:,:,:] = idx_arr[:-1,:,:]
                # set first index = 0
                resample[self.varname][0,:,:] = -9999.0
            return resample
        else:
            return combined_ds
            
                
        


In [4]:
class Output_T_P(object):
    
    def __init__(self,xr_ds,varname,yr_mo_lst):
        
        self.ds = xr_ds
        self.varname = varname
        self.yr_mo_lst = yr_mo_lst
        self.outputdir = '/glade/scratch/rossamower/snow/snowmodel/andy_sim/tuolumne/tuolumne_100m_exdistr/met/wrf/data/3_hrly/year/'
        self.toFile()
        
        
    def toFile(self):
        # # loop through months and output
        for var in [self.varname]:
            for item in self.yr_mo_lst:
                ms = item[1]
                yr = item[0]
                print("%s-%s" % (yr,ms))
                da_month = self.ds[var].sel(Time=slice("%s-%s" % (yr,ms),"%s-%s" % (yr,ms)))
                da_month.to_dataset().to_netcdf(self.outputdir + yr + '/month/' + ms + '/' + var + '.nc',
                                                encoding = {"Time":
                                                            {'dtype':'float64',
                                                             'units': 'hours since 1901-01-01 00:00:00',
                                                             'calendar':'standard'}})
                
class Output_RELH(object):
    
    def __init__(self,xr_Q2,xr_PSFC,xr_T2,varname,yr_mo_lst):
        
        self.ds = xr_Q2
        self.ds_PSFC = xr_PSFC
        self.ds_T2 = xr_T2
        self.varname = varname
        self.yr_mo_lst = yr_mo_lst
        self.outputdir = '/glade/scratch/rossamower/snow/snowmodel/andy_sim/tuolumne/tuolumne_100m_exdistr/met/wrf/data/3_hrly/year/'
        self.calcRELH()
        self.toFile()
        
    def calcRELH(self):
        es_T_0 = 6.11 # hPa # might need to multiply by 100!! because pressure is in Pa
        T_0 = 273.15 # K
        L = 2500000 # J/kg 
        Rw = 461.52 # J/kg*K
        
        self.ds['PSFC'] = self.ds_PSFC['PSFC']
        self.ds['T2'] = self.ds_T2['T2']

        ## calculate partial water vapor pressure (e)
        self.ds['e'] = (self.ds['Q2'] * self.ds['PSFC']) / (0.622 + (0.378*self.ds['Q2']))

        ## calculate saturation water vapor pressure (es)
        self.ds['e_s'] = es_T_0 * np.exp((L/Rw)*((1/T_0)-(1/self.ds['T2'])))

        ## caculate relative humidity
        self.ds['RELH_raw'] = self.ds['e'] / self.ds['e_s']

        ## clean relative humidity by setting values over 100 to 100
        conditions  = [ (self.ds['RELH_raw']<= 100.0),(self.ds['RELH_raw']> 100.0) ]
        choices     = [ self.ds['RELH_raw'],100.0 ]
        dir_array = np.select(conditions, choices, default=np.nan)
        
        ## check for nan values in array
        if len(np.argwhere(np.isnan(dir_array))) > 0:
            print("NAN VALUE FOUND")
            print(np.argwhere(np.isnan(dir_array)))
        
        self.ds['RELH'] = (('Time','south_north','west_east'),dir_array)
        
    def toFile(self):
        # # loop through months and output
        for var in [self.varname]:
            for item in self.yr_mo_lst:
                ms = item[1]
                yr = item[0]
                print("%s-%s" % (yr,ms))
                da_month = self.ds[var].sel(Time=slice("%s-%s" % (yr,ms),"%s-%s" % (yr,ms)))
                da_month.to_dataset().to_netcdf(self.outputdir + yr + '/month/' + ms + '/' + var + '.nc',
                                                encoding = {"Time":
                                                            {'dtype':'float64',
                                                             'units': 'hours since 1901-01-01 00:00:00',
                                                             'calendar':'standard'}})
                
class Output_wind(object):
    
    def __init__(self,xr_U10,xr_V10,varname,yr_mo_lst):
        
        self.ds = xr_U10
        self.ds_V10 = xr_V10
        self.varname = varname
        self.yr_mo_lst = yr_mo_lst
        self.outputdir = '/glade/scratch/rossamower/snow/snowmodel/andy_sim/tuolumne/tuolumne_100m_exdistr/met/wrf/data/3_hrly/year/'
        self.calcwind()
        self.toFile()
        
    def calcwind(self):
        ## add U10
        self.ds['V10'] = self.ds_V10['V10']

        ## calculate WSPD
        self.ds['WSPD']= (self.ds['V10']**2 + self.ds['U10']**2)**0.5

        ## calculate WDIR
        self.ds['V_U'] = abs(self.ds['V10']) / abs(self.ds['U10'])
        self.ds['WDEG'] = np.arctan(self.ds['V_U']) * 57.2958
        conditions  = [ ((self.ds['U10'] > 0.0) & (self.ds['V10'] > 0.0)), # Q1
                       ((self.ds['U10'] > 0.0) & (self.ds['V10'] < 0.0)),  # Q2
                       ((self.ds['U10'] < 0.0) & (self.ds['V10'] < 0.0)),  # Q3
                       ((self.ds['U10'] < 0.0) & (self.ds['V10'] > 0.0)), # Q4
                       ((self.ds['V10'] > 0.0) & (self.ds['U10'] == 0.0)), # 0
                       ((self.ds['V10'] == 0.0) & (self.ds['U10'] > 0.0)), # 90
                       ((self.ds['V10'] < 0.0) & (self.ds['U10'] == 0.0)),# 180
                       ((self.ds['V10'] == 0.0) & (self.ds['U10'] < 0.0)) # 270
                      ] 
## Remember direction is based on where wind is coming from. NOT WHERE IT IS COMING TO. 
## A positive U and V has a wind-direction between 180 and 270.
        choices     = [ 90.0 - self.ds['WDEG'] + 180, # Q1
                       90.0 + self.ds['WDEG'] + 180, # Q2
                       270.0 - self.ds['WDEG'] - 180.0, # Q3
                       270.0 + self.ds['WDEG'] - 180.0, # Q4
                      180.0,
                      270.0,
                      0.0,
                      90.0
                      ]
        dir_array = np.select(conditions, choices, default=np.nan)

        ## check for nan values in array
        if len(np.argwhere(np.isnan(dir_array))) > 0:
            print("NAN VALUE FOUND")
            print(np.argwhere(np.isnan(dir_array)))
    
        ## create 'wdir' array from dir_array
        self.ds['WDIR'] = (('Time','south_north','west_east'),dir_array)
        self.ds = self.ds.drop(['WDEG','V_U'])  
        print('Finished processing wind')
    
    def toFile(self):
        # # loop through months and output
        for var in ['WDIR','WSPD']:
            for item in self.yr_mo_lst:
                print(var)
                ms = item[1]
                yr = item[0]
                print("%s-%s" % (yr,ms))
                da_month = self.ds[var].sel(Time=slice("%s-%s" % (yr,ms),"%s-%s" % (yr,ms)))
                da_month.to_dataset().to_netcdf(self.outputdir + yr + '/month/' + ms + '/' + var + '.nc',
                                                encoding = {"Time":
                                                            {'dtype':'float64',
                                                             'units': 'hours since 1901-01-01 00:00:00',
                                                             'calendar':'standard'}})
                
        
        

In [5]:
"""RUN TEMPERATURE"""
inputvar = 'T2'
outputvar = inputvar
wateryr = 2010
hourly = 0

## create instance
temp = PreprocessNetCDF(inputvar,wateryr,hourly)
## create dataset
temp_ds = temp.concatenateNetCDF()


## write to directory
# Output_T_P(temp_ds,outputvar,temp.month_lst)



/glade/collections/rda/data/ds612.0/CTRL/2009/wrf2d_d01_CTRL_T2_200907-200909.nc
/glade/collections/rda/data/ds612.0/CTRL/2009/wrf2d_d01_CTRL_T2_200910-200912.nc
/glade/collections/rda/data/ds612.0/CTRL/2010/wrf2d_d01_CTRL_T2_201001-201003.nc
/glade/collections/rda/data/ds612.0/CTRL/2010/wrf2d_d01_CTRL_T2_201004-201006.nc
/glade/collections/rda/data/ds612.0/CTRL/2010/wrf2d_d01_CTRL_T2_201007-201009.nc
/glade/collections/rda/data/ds612.0/CTRL/2010/wrf2d_d01_CTRL_T2_201010-201012.nc


<xarray.Dataset>
Dimensions:  (Time: 13176, south_north: 1015, west_east: 1359)
Coordinates:
  * Time     (Time) datetime64[ns] 2009-07-01 ... 2010-12-31T23:00:00
    XLAT     (south_north, west_east) float32 18.14 18.15 18.15 ... 52.43 52.42
    XLONG    (south_north, west_east) float32 -122.9 -122.8 ... -57.0 -56.95
Dimensions without coordinates: south_north, west_east
Data variables:
    T2       (Time, south_north, west_east) float32 296.7 296.7 ... 271.0 271.1
    Times    (Time) |S19 b'2009-07-01_00:00:00' ... b'2010-12-31_23:00:00'
    XTIME    (Time) float32 4.601e+06 4.601e+06 ... 5.391e+06 5.391e+06
Attributes: (12/113)
    SIMULATION_START_DATE:           2000-10-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        1360
    SOUTH-NORTH_GRID_DIMENSION:      1016
    BOTTOM-TOP_GRID_DIMENSION:       51
    DX:                              4000.0
    DY:                              4000.0
    ...                              ...
    Division:                        NCAR/RAL/HAP
    history:                         Wed Feb  8 18:54:39 2017: ncks -O -4 /gl...
    NCO:                             "4.5.5"
    Publication_DOI:                 doi:10.1007/s00382-016-3327-9
    Conventions:                     CF-1.6
    TITLE:                            OUTPUT FROM WRF V3.4.1 MODEL , CURRENT ...

In [35]:
resample = temp_ds.resample(Time="3H").mean(dim="Time")
resample2 = temp_ds.resample(Time="3H",Ioffset = '1H').mean(dim="Time")
# temp_T2_slc = temp_ds.T2.sel(Time = slice("2009-09-30","2009-10-01"))
# temp_T2_slc.Time[23]

ValueError: Resampling only supported along single dimensions.

In [19]:
print(temp_ds.Time[0:23].values)
temp_ds.T2[0:23,0,0].values

['2009-07-01T00:00:00.000000000' '2009-07-01T01:00:00.000000000'
 '2009-07-01T02:00:00.000000000' '2009-07-01T03:00:00.000000000'
 '2009-07-01T04:00:00.000000000' '2009-07-01T05:00:00.000000000'
 '2009-07-01T06:00:00.000000000' '2009-07-01T07:00:00.000000000'
 '2009-07-01T08:00:00.000000000' '2009-07-01T09:00:00.000000000'
 '2009-07-01T10:00:00.000000000' '2009-07-01T11:00:00.000000000'
 '2009-07-01T12:00:00.000000000' '2009-07-01T13:00:00.000000000'
 '2009-07-01T14:00:00.000000000' '2009-07-01T15:00:00.000000000'
 '2009-07-01T16:00:00.000000000' '2009-07-01T17:00:00.000000000'
 '2009-07-01T18:00:00.000000000' '2009-07-01T19:00:00.000000000'
 '2009-07-01T20:00:00.000000000' '2009-07-01T21:00:00.000000000'
 '2009-07-01T22:00:00.000000000']


array([296.74652, 296.75143, 296.76175, 296.77023, 296.781  , 296.79187,
       296.80063, 296.70297, 296.65198, 296.5972 , 296.5461 , 296.49506,
       296.44196, 296.3806 , 296.31903, 296.25784, 296.199  , 296.14355,
       296.0868 , 296.09726, 296.10428, 296.11377, 296.11862],
      dtype=float32)

In [29]:
print(resample.Time[0:3].values)
print(resample.T2[0:3,0,0].values)
print('')
print('')
print(resample2.Time[0:3].values)
print(resample2.T2[0:3,0,0].values)
# temp_T2_slc = temp_ds.T2.sel(Time = slice("2009-09-30","2009-10-01"))
# temp_T2_slc.Time[23]

['2009-07-01T00:00:00.000000000' '2009-07-01T03:00:00.000000000'
 '2009-07-01T06:00:00.000000000']
[296.7532  296.78104 296.71854]


['2009-07-01T00:00:00.000000000' '2009-07-01T03:00:00.000000000'
 '2009-07-01T06:00:00.000000000']
[296.7566  296.78104 296.71854]


In [33]:
(296.75143+296.76175)/2

296.75659

In [ ]:
"""RUN PRECIPITATION"""
inputvar = 'PREC_ACC_NC'
outputvar = inputvar
wateryr = 2010
hourly = 0

## create instance
precip = PreprocessNetCDF(inputvar,wateryr,hourly)
## create dataset
precip_ds = precip.concatenateNetCDF()

## write to directory
Output_T_P(precip_ds,outputvar,precip.month_lst)

In [ ]:
"""RUN RELH"""
## input variables
inputQ2 = 'Q2'
inputPSFC = 'PSFC'
inputT2 = 'T2'
wateryr = 2010
hourly = 0
## output variable
outputvar = "RELH"

"""Q2"""
## create instance
Q2 = PreprocessNetCDF(inputQ2,wateryr,hourly)
## create dataset
Q2_ds = Q2.concatenateNetCDF()

"""PSFC"""
## create instance
PSFC = PreprocessNetCDF(inputPSFC,wateryr,hourly)
## create dataset
PSFC_ds = PSFC.concatenateNetCDF()

"""T2"""
## create instance
T2 = PreprocessNetCDF(inputT2,wateryr,hourly)
## create dataset
T2_ds = T2.concatenateNetCDF()

# create year month list
yr_mo_lst = Q2.month_lst
## write to directory
Output_RELH(Q2_ds,PSFC_ds,T2_ds,outputvar,yr_mo_lst)

In [ ]:
"""RUN WSPD AND WDIR"""
## input variables
inputU10 = 'U10'
inputV10 = 'V10'
wateryr = 2010
hourly = 0
## output variable
outputvar = ["WSPD","WDIR"]

"""U10"""
## create instance
U10 = PreprocessNetCDF(inputU10,wateryr,hourly)
## create dataset
U10_ds = U10.concatenateNetCDF()
print('Finished with U10')

"""V10"""
## create instance
V10 = PreprocessNetCDF(inputV10,wateryr,hourly)
## create dataset
V10_ds = V10.concatenateNetCDF()
print('Finished with V10')

# create year month list
yr_mo_lst = U10.month_lst
## write to directory
Output_wind(U10_ds,V10_ds,outputvar,yr_mo_lst)

In [ ]:
"""SAVE U10"""
## input variables
inputU10 = 'U10'
inputV10 = 'V10'
wateryr = 2012
hourly = 0
## output variable
outputvar = ["WSPD","WDIR"]

"""U10"""
## create instance
U10 = PreprocessNetCDF(inputU10,wateryr,hourly)
## create dataset
U10_ds = U10.concatenateNetCDF()
print('Start outputting')
Output_T_P(U10_ds,inputU10,U10.month_lst)
print('Finished with U10')



In [ ]:
"""SAVE U10"""
## input variables
inputU10 = 'U10'
inputV10 = 'V10'
wateryr = 2012
hourly = 0

"""V10"""
## create instance
V10 = PreprocessNetCDF(inputV10,wateryr,hourly)
## create dataset
V10_ds = V10.concatenateNetCDF()
print('Dataset created, begin output')
Output_T_P(V10_ds,inputV10,V10.month_lst)
print('Finished with V10')